In [3]:
import pandas as pd
import folium
import json
from pvlib import solarposition
import pandas as pd
import numpy as np
import math
from pyproj import Proj, transform
import pyproj
from shapely import geometry, ops
from shapely.geometry import mapping
import folium
import os
import cv2
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'folium'

In [4]:
building_points = [
            [
              45.415037, -75.701278              
            ],
            [
              45.414986, -75.701399              
            ],
            [
              45.414897, -75.701325              
            ],
            [
              45.414948, -75.701204              
            ],
            [
              45.415037, -75.701278              
            ]
          ]

In [ ]:
# Plotting Building on map

In [4]:
map1 = folium.Map(location=[ 45.415037, -75.701278], zoom_start=20, control_scale=True)

folium.Polygon(locations=building_points,color='orange',fill=True,opacity=0.3,fill_color='orange', fill_opacity=0.2).add_to(map1)    

map1
        

In [ ]:
# Getting sun's azimuth and elevation angle for particular lat, lon and date range 

In [11]:
times = pd.date_range('2020-09-28 14:00:00', '2020-09-28 15:30:00', closed='left', freq='30min')
dataframe = pd.DataFrame({'Date': times})  
dataframe.Date = dataframe.Date.dt.tz_localize('UTC').dt.tz_convert('America/Toronto')

latitude, longitude = 45.415037, -75.701278
solarpos = solarposition.get_solarposition(dataframe.Date, latitude, longitude)

# remove nighttime
solarpos = solarpos.loc[solarpos['apparent_elevation'] > 0, :]
print(solarpos)

                           apparent_zenith     zenith  apparent_elevation  \
Date                                                                        
2020-09-28 10:00:00-04:00        61.208188  61.238657           28.791812   
2020-09-28 10:30:00-04:00        57.323513  57.349668           32.676487   
2020-09-28 11:00:00-04:00        53.953079  53.976155           36.046921   

                           elevation     azimuth  equation_of_time  
Date                                                                
2020-09-28 10:00:00-04:00  28.761343  128.563931          9.548291  
2020-09-28 10:30:00-04:00  32.650332  136.020909          9.555256  
2020-09-28 11:00:00-04:00  36.023845  144.132022          9.562221  


In [ ]:
# shadow calculation with assumed height as 7

In [18]:
counter = 0
for index, row in solpos.iterrows():
    azimuth = row['azimuth']
    elevation = row['apparent_elevation']
    h=7
    shadowheight = h / math.tan(el * math.pi / 180)
    print(shadowheight)
    x = -1 * math.sin(math.radians(azimuth)) * abs(shadowheight)
    print(x)
    y =  -1 * math.cos(math.radians(azimuth)) * abs(shadowheight)
    print(y)
    
    map1 = folium.Map(location=[ 45.415037, -75.701278], zoom_start=20, control_scale=True)
    
    lines = []
    for point in building_points: 
        print(point[0],point[1])
        inProj = Proj(init='epsg:3857')
        outProj = Proj(init='epsg:4326')
        lat_x,lat_y = transform(outProj,inProj,point[0],point[1])
        print(lat_x,lat_y)
        new_x = x + lat_x
        new_y = y + lat_y
        print(new_x,new_y)
        new_lat,new_lon = transform(inProj,outProj,new_x,new_y)
        print(new_lat,new_lon)
        line =[]
        line.append((point[0],point[1]))
        line.append((new_lat,new_lon))
        lines.append(line)
        
    # combine them into a multi-linestring
    multi_line = geometry.MultiLineString(lines)
    p = mapping(multi_line.convex_hull)
    shaow_coordinates = p['coordinates']
    folium.Polygon(locations=shaow_coordinates, color='black',fill=True,opacity=0.1, fill_color='black', fill_opacity=0.5).add_to(map1)
    folium.Polygon(locations=building_points,color='orange',fill=True,opacity=0.3,fill_color='orange', fill_opacity=0.2).add_to(map1)
   
    date_time = str(solarpos.index.year[counter])+'-' + str(solarpos.index.month[counter])+'-' +str(solarpos.index.day[counter])+'-' + str(solpos.index.hour[counter])+'-' + str(solpos.index.minute[counter])+'-' +str(solpos.index.second[counter])
    name = 'shadow' + date_time + '.html'
    print(name)
    map1.save(name)
    counter=counter+1

12.737264104635326
-9.959433225236085
7.940251047875949
45.415037 -75.701278
5055578.7931976775 -13240980.762439694
5055568.8337644525 -13240972.822188646
45.414947532889116 -75.70126038346477
45.414986 -75.701399
5055573.115903648 -13241035.300686
5055563.156470423 -13241027.360434951
45.414896532889124 -75.70138138361075
45.414897 -75.701325
5055563.208468967 -13241001.94666351
5055553.249035742 -13240994.006412461
45.41480753288913 -75.70130738352148
45.414948 -75.701204
5055568.885762998 -13240947.40869357
5055558.926329773 -13240939.468442522
45.41485853288912 -75.7011863833755
45.415037 -75.701278
5055578.7931976775 -13240980.762439694
5055568.8337644525 -13240972.822188646
45.414947532889116 -75.70126038346477
shadow2020-9-28-10-0-0.html
10.913469513504579
-7.578267590960777
7.853259014064633
45.415037 -75.701278
5055578.7931976775 -13240980.762439694
5055571.214930086 -13240972.90918068
45.41496892326394 -75.70126057646864
45.414986 -75.701399
5055573.115903648 -13241035.300686